### Assessing regularization of Latin corpus texts
In Cythia Damon's masterful edition of Julius Caesar's _Bellum Civile_ she provides rules for the spelling regularizations she performed in an Appendix Orthographica. This notebook tries to quantify the quality and scope of these suggested regularizations when applied to a larger Latin corpus.

In [1]:
import os.path
from collections import Counter
import inspect
import os
import sys
from glob import glob
from typing import List, Dict, Tuple

from cltk.corpus.readers import get_corpus_reader
from cltk.stem.latin.j_v import JVReplacer
from cltk.tokenize.latin.sentence import SentenceTokenizer
from cltk.tokenize.word import WordTokenizer
from cltk.corpus.latin.phi5_index import PHI5_INDEX
from tqdm import tqdm

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)
 
from mlyoucanuse.aeoe_replacer import AEOEReplacer
from mlyoucanuse.text_cleaners import ( normalize_accents, disappear_angle_brackets,
                                        drop_punct, disappear_round_brackets,
                                         dehyphenate, accept_editorial,
                                        swallow_braces, swallow_obelized_words,
                                        swallow_square_brackets)

# Latin Prefix Regularizations
Adapted from the Appendix Orthographica of Cythia Damon _C. Iuli Caesaris commentarii de bello civili_ (Bellum civile, or Civil War) (Oxford Classical Texts), pp. 166

In [2]:
# The first string in the tuple is the preferred prefix
# The second string in the tuple is the anomaly to be regularized
regularizations = [('adf', 'aff'),
                  ('agg', 'adg'), 
                  ('all', 'adl'),
                  ('adm', 'amm'),
                  ('adn', 'ann'), # problematic because of 'annuo', '->', 'adnuo' viz. annuus,a,um
                  ('app', 'adp'),
                  ('arr', 'adr'), 
                  ('ass', 'ads'), 
                  ('adst', 'asstan'),
                  ('att', 'adt'), 
                  ('afu', 'affu'), 
                  ('afu', 'abfu'), 
                  ('conic', 'coic'),
                  ('conic', 'conit'), 
                  # ('conic', 'coit'),  # problematic with: ('coitus', '->', 'collus')
                  ('coll', 'conl'), 
                  ('comp', 'conp'),
                  ('corr', 'conr'), 
                  ('ill', 'inl'),
                  ('imm', 'inm'),
                  ('imp', 'inp'), 
                  ('irr', 'inr'),
                  ('obt', 'opt'),
                  ('opp', 'obp'),
                  ('prae', 'pre'),
                  ('subm', 'summ'),
                  ('suff', 'subf'), 
                  ('supp', 'subp'),
                  ('trai', 'transi') # problematic: ('transis', '->', 'trais', 7)
                 ]

In [3]:
# In the following sections, we will show Damon's preferred prefix first

# some useful functions

def get_matches(prefix:str, word_counts:Dict[str,int])->Tuple[int, List[str]]:
    egs = [(key, val) for key,val in word_counts.items() if key.startswith(prefix)]
    if len(egs) ==0:
        return 0, []
    vals, counts = zip(*egs)
    egs = sorted(egs, key=lambda x:x[1], reverse=True)
    return sum(counts), egs

def print_regularizations(word_counts):
    total_transformations = 0
    for prefix_one, prefix_two in regularizations:
        prefix_one_num, prefix_one_egs = get_matches(prefix_one, word_counts) 
        prefix_two_num, prefix_two_egs = get_matches(prefix_two, word_counts) 
        print(f"# prefix examples: {prefix_one}- {prefix_one_num:,}, {prefix_two}- {prefix_two_num:,}")
        common_transforms = []
        for word, count in prefix_two_egs:
            transformed = word.replace(prefix_two, prefix_one)
            if transformed in word_counts:
                common_transforms.append((word, '->', transformed, count))
                total_transformations +=count
        print(f'exact transform matches: {len(common_transforms)}')            
        for item in common_transforms:
            print(item)
    print(f"total transformations proposed {total_transformations:,}")        

# Tesserae Corpus: Proposed Spelling Regularizations

In [4]:
tesserae = get_corpus_reader(corpus_name='latin_text_tesserae', language='latin')
word_counts = Counter()
jv_replacer = JVReplacer()
aeoe_replacer = AEOEReplacer()
toker = WordTokenizer('latin')
sent_toker = SentenceTokenizer()

for file in tqdm(tesserae.fileids(), total=len(tesserae.fileids())):
    for sent in tesserae.sents(file):
        sent = aeoe_replacer.replace(jv_replacer.replace(sent))
        sent = normalize_accents(sent)
        sent = accept_editorial(sent)
        sent = drop_punct(sent)
        for token in toker.tokenize(sent):
            word_counts.update({token : 1})

print_regularizations(word_counts)

100%|██████████| 762/762 [03:24<00:00,  3.72it/s]


# prefix examples: adf- 4,606, aff- 1,488
exact transform matches: 257
('affectus', '->', 'adfectus', 144)
('affectu', '->', 'adfectu', 70)
('affectum', '->', 'adfectum', 64)
('affert', '->', 'adfert', 40)
('afferre', '->', 'adferre', 39)
('affatim', '->', 'adfatim', 33)
('affirmat', '->', 'adfirmat', 32)
('affectibus', '->', 'adfectibus', 28)
('affectio', '->', 'adfectio', 25)
('affici', '->', 'adfici', 21)
('afferat', '->', 'adferat', 21)
('afferri', '->', 'adferri', 20)
('afferunt', '->', 'adferunt', 17)
('afflictis', '->', 'adflictis', 16)
('afflicti', '->', 'adflicti', 15)
('affuit', '->', 'adfuit', 15)
('affirmauit', '->', 'adfirmauit', 15)
('afficit', '->', 'adficit', 15)
('affore', '->', 'adfore', 14)
('affatur', '->', 'adfatur', 14)
('affirmare', '->', 'adfirmare', 13)
('affinitatem', '->', 'adfinitatem', 13)
('afflixit', '->', 'adflixit', 12)
('afflatu', '->', 'adflatu', 12)
('afferret', '->', 'adferret', 11)
('affines', '->', 'adfines', 11)
('affecti', '->', 'adfecti', 10)
(

# prefix examples: all- 1,249, adl- 873
exact transform matches: 94
('adlatum', '->', 'allatum', 53)
('adloquitur', '->', 'alloquitur', 48)
('adlata', '->', 'allata', 38)
('adlatae', '->', 'allatae', 35)
('adlocutus', '->', 'allocutus', 26)
('adloqui', '->', 'alloqui', 23)
('adluit', '->', 'alluit', 21)
('adloquio', '->', 'alloquio', 19)
('adluitur', '->', 'alluitur', 16)
('adligat', '->', 'alligat', 13)
('adloquiis', '->', 'alloquiis', 13)
('adloquor', '->', 'alloquor', 13)
('adlatis', '->', 'allatis', 12)
('adlatura', '->', 'allatura', 12)
('adlatas', '->', 'allatas', 12)
('adloquar', '->', 'alloquar', 11)
('adludens', '->', 'alludens', 10)
('adlatus', '->', 'allatus', 10)
('adlato', '->', 'allato', 10)
('adlatam', '->', 'allatam', 9)
('adlaturum', '->', 'allaturum', 8)
('adlicere', '->', 'allicere', 8)
('adleuat', '->', 'alleuat', 8)
('adlicit', '->', 'allicit', 7)
('adlegit', '->', 'allegit', 7)
('adlocutio', '->', 'allocutio', 7)
('adligauit', '->', 'alligauit', 5)
('adludit', '->

# prefix examples: arr- 535, adr- 305
exact transform matches: 49
('adrogantia', '->', 'arrogantia', 26)
('adrogantiam', '->', 'arrogantiam', 24)
('adrogans', '->', 'arrogans', 19)
('adridens', '->', 'arridens', 18)
('adrogantiae', '->', 'arrogantiae', 11)
('adroganter', '->', 'arroganter', 11)
('adrepto', '->', 'arrepto', 8)
('adrogantius', '->', 'arrogantius', 6)
('adripuit', '->', 'arripuit', 6)
('adridet', '->', 'arridet', 6)
('adripere', '->', 'arripere', 5)
('adrectis', '->', 'arrectis', 5)
('adrogare', '->', 'arrogare', 4)
('adrogantis', '->', 'arrogantis', 4)
('adrisit', '->', 'arrisit', 4)
('adrogo', '->', 'arrogo', 4)
('adroganti', '->', 'arroganti', 4)
('adridere', '->', 'arridere', 4)
('adripiunt', '->', 'arripiunt', 3)
('adrogantes', '->', 'arrogantes', 3)
('adrogantem', '->', 'arrogantem', 3)
('adripit', '->', 'arripit', 3)
('adrexit', '->', 'arrexit', 3)
('adrectae', '->', 'arrectae', 3)
('adroget', '->', 'arroget', 2)
('adrectus', '->', 'arrectus', 2)
('adrectos', '->',

# prefix examples: adst- 503, asstan- 0
exact transform matches: 0
# prefix examples: att- 3,163, adt- 775
exact transform matches: 173
('adtende', '->', 'attende', 43)
('adtulit', '->', 'attulit', 42)
('adtendite', '->', 'attendite', 29)
('adtulerunt', '->', 'attulerunt', 23)
('adtinet', '->', 'attinet', 20)
('adtribuit', '->', 'attribuit', 15)
('adtributa', '->', 'attributa', 15)
('adtollere', '->', 'attollere', 14)
('adtulerat', '->', 'attulerat', 14)
('adtingere', '->', 'attingere', 14)
('adtendere', '->', 'attendere', 13)
('adtrita', '->', 'attrita', 11)
('adtributae', '->', 'attributae', 9)
('adtritis', '->', 'attritis', 9)
('adtrahere', '->', 'attrahere', 7)
('adtendit', '->', 'attendit', 7)
('adtingit', '->', 'attingit', 6)
('adtonitus', '->', 'attonitus', 6)
('adtendat', '->', 'attendat', 6)
('adtrectare', '->', 'attrectare', 6)
('adtuli', '->', 'attuli', 6)
('adtenuatus', '->', 'attenuatus', 6)
('adtoniti', '->', 'attoniti', 6)
('adtingat', '->', 'attingat', 6)
('adtendis', '

# prefix examples: comp- 7,071, conp- 3,145
exact transform matches: 531
('conposuit', '->', 'composuit', 54)
('conplexus', '->', 'complexus', 49)
('conparatio', '->', 'comparatio', 48)
('conprehendit', '->', 'comprehendit', 47)
('conpleuit', '->', 'compleuit', 42)
('conpertum', '->', 'compertum', 40)
('conpositis', '->', 'compositis', 37)
('conposita', '->', 'composita', 35)
('conpulsi', '->', 'compulsi', 35)
('conpositum', '->', 'compositum', 34)
('conprehendi', '->', 'comprehendi', 34)
('conplures', '->', 'complures', 33)
('conpulit', '->', 'compulit', 33)
('conpositio', '->', 'compositio', 32)
('conponere', '->', 'componere', 31)
('conperta', '->', 'comperta', 31)
('conposito', '->', 'composito', 29)
('conplexu', '->', 'complexu', 29)
('conparationem', '->', 'comparationem', 25)
('conprehendere', '->', 'comprehendere', 25)
('conplecti', '->', 'complecti', 23)
('conpulsus', '->', 'compulsus', 23)
('conprehensum', '->', 'comprehensum', 23)
('conplexa', '->', 'complexa', 22)
('conplet

# prefix examples: ill- 45,257, inl- 2,537
exact transform matches: 172
('inlita', '->', 'illita', 146)
('inlinitur', '->', 'illinitur', 125)
('inlitum', '->', 'illitum', 109)
('inlustris', '->', 'illustris', 70)
('inlinuntur', '->', 'illinuntur', 65)
('inlata', '->', 'illata', 59)
('inlecebris', '->', 'illecebris', 53)
('inlustres', '->', 'illustres', 50)
('inlustri', '->', 'illustri', 46)
('inlustribus', '->', 'illustribus', 38)
('inluxit', '->', 'illuxit', 35)
('inlustrium', '->', 'illustrium', 35)
('inlatum', '->', 'illatum', 35)
('inlustrem', '->', 'illustrem', 33)
('inlustre', '->', 'illustre', 32)
('inlitis', '->', 'illitis', 23)
('inlecebras', '->', 'illecebras', 22)
('inlustrior', '->', 'illustrior', 22)
('inludere', '->', 'illudere', 22)
('inluminat', '->', 'illuminat', 19)
('inlatus', '->', 'illatus', 19)
('inlicitum', '->', 'illicitum', 18)
('inlustrat', '->', 'illustrat', 17)
('inlustrata', '->', 'illustrata', 17)
('inlato', '->', 'illato', 15)
('inlatam', '->', 'illatam',

('inpediar', '->', 'impediar', 1)
('inpediendus', '->', 'impediendus', 1)
('inperuius', '->', 'imperuius', 1)
('inplerant', '->', 'implerant', 1)
('inplerat', '->', 'implerat', 1)
('inpluuium', '->', 'impluuium', 1)
('inpransum', '->', 'impransum', 1)
('inportunam', '->', 'importunam', 1)
('inpluat', '->', 'impluat', 1)
('inplexi', '->', 'implexi', 1)
('inpauidae', '->', 'impauidae', 1)
('inpactos', '->', 'impactos', 1)
('inprimens', '->', 'imprimens', 1)
('inpellenti', '->', 'impellenti', 1)
('inponendo', '->', 'imponendo', 1)
('inprobatus', '->', 'improbatus', 1)
('inponendos', '->', 'imponendos', 1)
('inpugnari', '->', 'impugnari', 1)
('inperitans', '->', 'imperitans', 1)
('inpellentia', '->', 'impellentia', 1)
('inplexum', '->', 'implexum', 1)
('inpellendos', '->', 'impellendos', 1)
('inprobiores', '->', 'improbiores', 1)
('inprouidae', '->', 'improuidae', 1)
('inpertiuit', '->', 'impertiuit', 1)
('inprobatam', '->', 'improbatam', 1)
('inploranda', '->', 'imploranda', 1)
('inpletis

# prefix examples: obt- 2,286, opt- 4,633
exact transform matches: 95
('optinent', '->', 'obtinent', 27)
('optinuit', '->', 'obtinuit', 21)
('optinet', '->', 'obtinet', 20)
('optigit', '->', 'obtigit', 19)
('optinere', '->', 'obtinere', 14)
('optulit', '->', 'obtulit', 11)
('optemperare', '->', 'obtemperare', 7)
('optinuere', '->', 'obtinuere', 6)
('optineat', '->', 'obtineat', 6)
('optinente', '->', 'obtinente', 6)
('optineri', '->', 'obtineri', 5)
('optinens', '->', 'obtinens', 5)
('optinebat', '->', 'obtinebat', 4)
('optutu', '->', 'obtutu', 3)
('optutum', '->', 'obtutum', 3)
('optines', '->', 'obtines', 3)
('optineret', '->', 'obtineret', 3)
('optineant', '->', 'obtineant', 3)
('optingere', '->', 'obtingere', 3)
('optinuisset', '->', 'obtinuisset', 3)
('optorto', '->', 'obtorto', 3)
('optigerat', '->', 'obtigerat', 3)
('optinendi', '->', 'obtinendi', 3)
('optulerat', '->', 'obtulerat', 3)
('optutus', '->', 'obtutus', 2)
('optinentes', '->', 'obtinentes', 2)
('optuli', '->', 'obtuli

# prefix examples: suff- 1,811, subf- 48
exact transform matches: 18
('subfixa', '->', 'suffixa', 3)
('subfusis', '->', 'suffusis', 3)
('subferuefacta', '->', 'sufferuefacta', 3)
('subfossis', '->', 'suffossis', 2)
('subferuefactis', '->', 'sufferuefactis', 2)
('subferuefactum', '->', 'sufferuefactum', 2)
('subfragabatur', '->', 'suffragabatur', 1)
('subfodiat', '->', 'suffodiat', 1)
('subfusiones', '->', 'suffusiones', 1)
('subfusae', '->', 'suffusae', 1)
('subfusus', '->', 'suffusus', 1)
('subfundit', '->', 'suffundit', 1)
('subfodit', '->', 'suffodit', 1)
('subfundere', '->', 'suffundere', 1)
('subfuso', '->', 'suffuso', 1)
('subfodere', '->', 'suffodere', 1)
('subfundi', '->', 'suffundi', 1)
('subfragatio', '->', 'suffragatio', 1)
# prefix examples: supp- 2,879, subp- 144
exact transform matches: 53
('subplantat', '->', 'supplantat', 5)
('subposuit', '->', 'supposuit', 4)
('subposito', '->', 'supposito', 4)
('subponere', '->', 'supponere', 3)
('subpositis', '->', 'suppositis', 2)
(

In [5]:
# when coerce to lower: total transformations proposed 26,088

# Other regularizations worth considering: archaisms

In [6]:
word_counts.get('quom') # ~ cum

471

# Perhaps transliterated words should appear in the original Greek?

In [7]:
word_counts.get('alla'), word_counts.get('allh'), word_counts.get('kai')   

(147, 56, 1552)

# Phi5 Corpus: Proposed Spelling Regularizations

In [8]:
author_index = {val:key for key,val in PHI5_INDEX.items()}  

def get_phi5_author_files(author_name, author_index):
    stub = author_index[author_name]
    return glob(os.path.expanduser(f'~/cltk_data/latin/text/phi5/individual_works/{stub}*.txt'))

word_counts = Counter()
jv_replacer = JVReplacer()
aeoe_replacer = AEOEReplacer()
toker = WordTokenizer('latin')
sent_toker = SentenceTokenizer() 

for author in tqdm(author_index):
    files = get_phi5_author_files(author, author_index)
    for file in files:
        with open(file, 'rt') as fin:
            text = fin.read()
            text = text.replace("-\n", "") # Phi5 has some hyphenated line endings
            text = text.replace("\n", " ")
            text = aeoe_replacer.replace(jv_replacer.replace(text))
            for sent in sent_toker.tokenize(text):
                sent = dehyphenate(sent) # because it's Phi5
                sent = swallow_braces(sent)
                sent = swallow_square_brackets(sent)
                sent = disappear_round_brackets(sent)
                sent = swallow_obelized_words(sent)
                sent = disappear_angle_brackets(sent)            
                sent = drop_punct(sent)
                sent = normalize_accents(sent)            
                for word in toker.tokenize(sent):
                    if word.isnumeric():
                        continue
                    word_counts.update({word : 1}) 

print_regularizations(word_counts)

100%|██████████| 362/362 [02:51<00:00,  2.12it/s]


# prefix examples: adf- 5,232, aff- 964
exact transform matches: 206
('affectus', '->', 'adfectus', 107)
('affectu', '->', 'adfectu', 58)
('affectum', '->', 'adfectum', 36)
('affert', '->', 'adfert', 33)
('affectio', '->', 'adfectio', 30)
('afferre', '->', 'adferre', 25)
('afferat', '->', 'adferat', 24)
('affectibus', '->', 'adfectibus', 20)
('affirmauit', '->', 'adfirmauit', 18)
('affectionem', '->', 'adfectionem', 17)
('afferri', '->', 'adferri', 16)
('affatim', '->', 'adfatim', 15)
('affectionis', '->', 'adfectionis', 14)
('afferunt', '->', 'adferunt', 13)
('affici', '->', 'adfici', 13)
('afferret', '->', 'adferret', 13)
('affecti', '->', 'adfecti', 12)
('affirmat', '->', 'adfirmat', 11)
('afflictis', '->', 'adflictis', 9)
('afflicti', '->', 'adflicti', 9)
('affuit', '->', 'adfuit', 8)
('affertur', '->', 'adfertur', 7)
('affirmant', '->', 'adfirmant', 7)
('afferet', '->', 'adferet', 7)
('affluens', '->', 'adfluens', 7)
('afflixit', '->', 'adflixit', 7)
('affero', '->', 'adfero', 7)


# prefix examples: all- 921, adl- 981
exact transform matches: 91
('adloquitur', '->', 'alloquitur', 78)
('adlatum', '->', 'allatum', 37)
('adlata', '->', 'allata', 32)
('adlatae', '->', 'allatae', 31)
('adluit', '->', 'alluit', 31)
('adluitur', '->', 'alluitur', 29)
('adloqui', '->', 'alloqui', 27)
('adlocutus', '->', 'allocutus', 23)
('adludit', '->', 'alludit', 22)
('adloquio', '->', 'alloquio', 18)
('adloquor', '->', 'alloquor', 17)
('adlusit', '->', 'allusit', 16)
('adlatis', '->', 'allatis', 13)
('adlato', '->', 'allato', 13)
('adlatura', '->', 'allatura', 12)
('adlatas', '->', 'allatas', 12)
('adlatus', '->', 'allatus', 12)
('adloquar', '->', 'alloquar', 12)
('adloquiis', '->', 'alloquiis', 11)
('adlatam', '->', 'allatam', 10)
('adlicere', '->', 'allicere', 9)
('adligat', '->', 'alligat', 8)
('adlocutio', '->', 'allocutio', 8)
('adleuare', '->', 'alleuare', 8)
('adloquendi', '->', 'alloquendi', 8)
('adlaturum', '->', 'allaturum', 7)
('adleuat', '->', 'alleuat', 6)
('adligatum', 

# prefix examples: arr- 425, adr- 506
exact transform matches: 46
('adrogantia', '->', 'arrogantia', 35)
('adrogantiam', '->', 'arrogantiam', 28)
('adrogans', '->', 'arrogans', 27)
('adrogantiae', '->', 'arrogantiae', 19)
('adridens', '->', 'arridens', 15)
('adroganter', '->', 'arroganter', 14)
('adrogatio', '->', 'arrogatio', 14)
('adrepto', '->', 'arrepto', 10)
('adrogantius', '->', 'arrogantius', 10)
('adrogare', '->', 'arrogare', 8)
('adrogantem', '->', 'arrogantem', 7)
('adrepta', '->', 'arrepta', 6)
('adripere', '->', 'arripere', 5)
('adripuit', '->', 'arripuit', 5)
('adreptum', '->', 'arreptum', 5)
('adroget', '->', 'arroget', 5)
('adrogo', '->', 'arrogo', 4)
('adrogantis', '->', 'arrogantis', 4)
('adrisit', '->', 'arrisit', 4)
('adridere', '->', 'arridere', 4)
('adridet', '->', 'arridet', 3)
('adrisisset', '->', 'arrisisset', 3)
('adrexit', '->', 'arrexit', 3)
('adrectis', '->', 'arrectis', 3)
('adrogat', '->', 'arrogat', 3)
('adripiunt', '->', 'arripiunt', 2)
('adripi', '->', 

# prefix examples: adst- 472, asstan- 0
exact transform matches: 0
# prefix examples: att- 3,357, adt- 631
exact transform matches: 169
('adtinet', '->', 'attinet', 25)
('adtulit', '->', 'attulit', 24)
('adtributa', '->', 'attributa', 22)
('adtribuit', '->', 'attribuit', 18)
('adtingere', '->', 'attingere', 16)
('adtingit', '->', 'attingit', 16)
('adtendere', '->', 'attendere', 15)
('adtributae', '->', 'attributae', 10)
('adtulerat', '->', 'attulerat', 8)
('adtributum', '->', 'attributum', 8)
('adtritu', '->', 'attritu', 8)
('adtingat', '->', 'attingat', 8)
('adtrahere', '->', 'attrahere', 8)
('adtritis', '->', 'attritis', 8)
('adtende', '->', 'attende', 8)
('adtulisse', '->', 'attulisse', 7)
('adtulerit', '->', 'attulerit', 6)
('adtemptare', '->', 'attemptare', 6)
('adtulerunt', '->', 'attulerunt', 5)
('adtinent', '->', 'attinent', 5)
('adtributas', '->', 'attributas', 5)
('adtingunt', '->', 'attingunt', 5)
('adtulisset', '->', 'attulisset', 5)
('adtributi', '->', 'attributi', 5)
('ad

...

# prefix examples: corr- 2,410, conr- 141
exact transform matches: 59
('conrumpere', '->', 'corrumpere', 6)
('conroborat', '->', 'corroborat', 4)
('conrupit', '->', 'corrupit', 4)
('conrupta', '->', 'corrupta', 4)
('conrumpitur', '->', 'corrumpitur', 3)
('conrumpet', '->', 'corrumpet', 3)
('conruit', '->', 'corruit', 3)
('conrupti', '->', 'corrupti', 3)
('conruant', '->', 'corruant', 2)
('conroborata', '->', 'corroborata', 2)
('conroborari', '->', 'corroborari', 2)
('conreptus', '->', 'correptus', 2)
('conrumpunt', '->', 'corrumpunt', 2)
('conruerunt', '->', 'corruerunt', 2)
('conrumpit', '->', 'corrumpit', 2)
('conrupisse', '->', 'corrupisse', 2)
('conruptis', '->', 'corruptis', 2)
('conruperat', '->', 'corruperat', 2)
('conroboret', '->', 'corroboret', 1)
('conruptorem', '->', 'corruptorem', 1)
('conrepsit', '->', 'correpsit', 1)
('conruerint', '->', 'corruerint', 1)
('conrumpis', '->', 'corrumpis', 1)
('conruget', '->', 'corruget', 1)
('conripiant', '->', 'corripiant', 1)
('conrepta

exact transform matches: 814
('inpune', '->', 'impune', 143)
('inposita', '->', 'imposita', 139)
('inposuit', '->', 'imposuit', 117)
('inponitur', '->', 'imponitur', 117)
('inpositum', '->', 'impositum', 116)
('inprudens', '->', 'imprudens', 63)
('inponere', '->', 'imponere', 61)
('inprouiso', '->', 'improuiso', 59)
('inpatiens', '->', 'impatiens', 57)
('inpia', '->', 'impia', 55)
('inponuntur', '->', 'imponuntur', 54)
('inponi', '->', 'imponi', 54)
('inpulsu', '->', 'impulsu', 53)
('inposito', '->', 'imposito', 51)
('inprobus', '->', 'improbus', 49)
('inprobe', '->', 'improbe', 49)
('inpulit', '->', 'impulit', 48)
('inpensa', '->', 'impensa', 47)
('inpar', '->', 'impar', 42)
('inpositis', '->', 'impositis', 41)
('inpositus', '->', 'impositus', 41)
('inpatientia', '->', 'impatientia', 38)
('inpendio', '->', 'impendio', 37)
('inponendum', '->', 'imponendum', 37)
('inponit', '->', 'imponit', 36)
('inproba', '->', 'improba', 36)
('inpleuit', '->', 'impleuit', 34)
('inponunt', '->', 'impon

# prefix examples: irr- 403, inr- 1,412
exact transform matches: 125
('inrita', '->', 'irrita', 112)
('inritum', '->', 'irritum', 45)
('inrupit', '->', 'irrupit', 44)
('inrumpere', '->', 'irrumpere', 32)
('inrumpit', '->', 'irrumpit', 29)
('inritus', '->', 'irritus', 28)
('inridere', '->', 'irridere', 21)
('inritat', '->', 'irritat', 19)
('inritata', '->', 'irritata', 18)
('inrigat', '->', 'irrigat', 17)
('inritas', '->', 'irritas', 16)
('inruit', '->', 'irruit', 15)
('inruptio', '->', 'irruptio', 14)
('inridens', '->', 'irridens', 14)
('inrito', '->', 'irrito', 14)
('inrupere', '->', 'irrupere', 13)
('inritatus', '->', 'irritatus', 13)
('inritatis', '->', 'irritatis', 13)
('inrisio', '->', 'irrisio', 12)
('inritare', '->', 'irritare', 12)
('inritam', '->', 'irritam', 12)
('inrumperent', '->', 'irrumperent', 11)
('inrepit', '->', 'irrepit', 11)
('inritant', '->', 'irritant', 11)
('inriti', '->', 'irriti', 11)
('inritet', '->', 'irritet', 10)
('inridet', '->', 'irridet', 9)
('inritae', 

# prefix examples: opp- 6,050, obp- 134
exact transform matches: 59
('obprobrium', '->', 'opprobrium', 14)
('obprobrio', '->', 'opprobrio', 6)
('obpugnationem', '->', 'oppugnationem', 6)
('obpugnatio', '->', 'oppugnatio', 5)
('obpressus', '->', 'oppressus', 5)
('obpugnare', '->', 'oppugnare', 4)
('obposuit', '->', 'opposuit', 4)
('obprimere', '->', 'opprimere', 4)
('obpugnaret', '->', 'oppugnaret', 3)
('obpugnanda', '->', 'oppugnanda', 2)
('obpugnari', '->', 'oppugnari', 2)
('obpressi', '->', 'oppressi', 2)
('obpugnabat', '->', 'oppugnabat', 2)
('obpugnatum', '->', 'oppugnatum', 2)
('obprobria', '->', 'opprobria', 2)
('obposita', '->', 'opposita', 2)
('obpugnatoribus', '->', 'oppugnatoribus', 2)
('obpositam', '->', 'oppositam', 2)
('obpugnatores', '->', 'oppugnatores', 2)
('obpugnantium', '->', 'oppugnantium', 2)
('obpugnabant', '->', 'oppugnabant', 2)
('obpugnauerunt', '->', 'oppugnauerunt', 2)
('obpugnabimus', '->', 'oppugnabimus', 1)
('obpugnatur', '->', 'oppugnatur', 1)
('obpugna',

('subprimenda', '->', 'supprimenda', 1)
('subplantare', '->', 'supplantare', 1)
('subpone', '->', 'suppone', 1)
('subpeditet', '->', 'suppeditet', 1)
('subpeditare', '->', 'suppeditare', 1)
('subpeditabatur', '->', 'suppeditabatur', 1)
('subpressis', '->', 'suppressis', 1)
('subparum', '->', 'supparum', 1)
('subpositas', '->', 'suppositas', 1)
('subpressae', '->', 'suppressae', 1)
('subplantat', '->', 'supplantat', 1)
# prefix examples: trai- 680, transi- 2,085
exact transform matches: 7
('transi', '->', 'trai', 12)
('transiectio', '->', 'traiectio', 2)
('transicit', '->', 'traicit', 1)
('transiecit', '->', 'traiecit', 1)
('transiectionem', '->', 'traiectionem', 1)
('transiectos', '->', 'traiectos', 1)
('transiecti', '->', 'traiecti', 1)
total transformations proposed 25,020


In [9]:
# Other regularizations possible, such as:
word_counts.get('quom') # archaic form of 'cum'

731